## Rubric

Instructions: DELETE this cell before you submit via a `git push` to your repo before deadline. This cell is for your reference only and is not needed in your report. 

Scoring: Out of 10 points

- Each Developing  => -2 pts
- Each Unsatisfactory/Missing => -4 pts
  - until the score is 

If students address the detailed feedback in a future checkpoint they will earn these points back


|                  | Unsatisfactory                                                                                                                                                                                                    | Developing                                                                                                                                                                                              | Proficient                                     | Excellent                                                                                                                              |
|------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------|
| Data relevance   | Did not have data relevant to their question. Or the datasets don't work together because there is no way to line them up against each other. If there are multiple datasets, most of them have this trouble | Data was only tangentially relevant to the question or a bad proxy for the question. If there are multiple datasets, some of them may be irrelevant or can't be easily combined.                       | All data sources are relevant to the question. | Multiple data sources for each aspect of the project. It's clear how the data supports the needs of the project.                         |
| Data description | Dataset or its cleaning procedures are not described. If there are multiple datasets, most have this trouble                                                                                              | Data was not fully described. If there are multiple datasets, some of them are not fully described                                                                                                      | Data was fully described                       | The details of the data descriptions and perhaps some very basic EDA also make it clear how the data supports the needs of the project. |
| Data wrangling   | Did not obtain data. They did not clean/tidy the data they obtained.  If there are multiple datasets, most have this trouble                                                                                 | Data was partially cleaned or tidied. Perhaps you struggled to verify that the data was clean because they did not present it well. If there are multiple datasets, some have this trouble | The data is cleaned and tidied.                | The data is spotless and they used tools to visualize the data cleanliness and you were convinced at first glance                      |


# COGS 108 - Data Checkpoint

## Authors

**Ali Ahmed**: Background research, Writing - original draft

**Rodayna Alnaggar**: Background research, Data curation

**Tessa Kibbe**: Conceptualization, Writing - review & editing

**Sabine A Sanchez**: Data curation, Methodology

**Maanav R Singh**: Project administration, Writing - review & editing

## Research Question

Has the introduction of Google AI Overviews in 2024 led to a statistically significant shift in the proportions of search behaviors (organic clicks vs. zero-click searches) for informational queries between 2023 and 2025?


## Background and Prior Work

The way we search for information online is going through a huge change right now. For about 30 years, Google and other search engines worked pretty simply: you typed in a question, and they gave you a list of websites to click on. This "Ten Blue Links" model meant that users had to do the work of clicking through different sites, reading them, and figuring out what was true on their own. But now, with AI being added to search engines, things are different. Instead of just showing you where to find answers, search engines are now trying to give you the answer directly. Researchers call this shift moving from "Information Retrieval" to "Generative Information Retrieval."

This change didn't happen overnight, it built up over several years. In 2015, Google started using RankBrain; in 2019 they added BERT; in 2023 they launched the Search Generative Experience (SGE), which became "AI Overviews" in 2024. This was the first time Google actually showed AI-generated text at the top of search results instead of just links.

One major consequence is what researchers call the "Great Decoupling"—more people are searching than ever, but fewer people are actually clicking through to websites. Industry studies (e.g., Seer Interactive, Ahrefs) report that AI Overviews reduce organic click-through rates by roughly 50–60%. We are using Wikipedia Clickstream and Google Trends to test whether the introduction of AI Overviews is associated with a measurable shift in click-through behavior for informational queries.

## Hypothesis


We hypothesize that the rollout of Google AI Overviews in 2024 has caused a statistically significant shift in search behavior for informational queries, characterized by an increase in zero-click search rates and a corresponding decrease in organic click-through rates. We expect to see this change in proportions specifically in informational queries compared to navigational and transactional queries because they seek to find answers rather than destinations.

In addition, we hypothesize that the impact of AI Overviews on informational search behavior increased zero-click searches in a nonlinear pattern: modest changes immediately after rollout (lagged adoption), followed by a rapid increase as users grew more accustomed to relying on AI-generated summaries. This prediction is based on industry research showing that AI Overviews reduce organic clicks by 50–60% and the broader trend toward zero-click searches, suggesting users are increasingly accepting AI-generated answers without clicking through to verify information from original sources.

## Data

### Data overview

We use two datasets to proxy zero-click behavior: Wikipedia Clickstream (clicks from search to Wikipedia) and Google Trends (search interest over time). If search interest stays stable but clicks from Google to Wikipedia drop after AI Overviews, that gap supports an increase in zero-click behavior.

- **Dataset #1 – Wikipedia Clickstream**
  - **Dataset Name:** Wikimedia Clickstream (English Wikipedia, monthly)
  - **Link:** https://dumps.wikimedia.org/other/clickstream/
  - **Observations:** One row per (referrer, destination article) pair per month; millions of pairs per monthly file.
  - **Variables:** `prev` (referrer: e.g. "other-search" for search engines, or article title), `curr` (destination article), `type` (link/external/other), `n` (count of that transition).
  - **Relevance:** Filtering `prev = 'other-search'` (and/or search-related referrers) gives monthly counts of clicks from search engines to Wikipedia articles. A decline in these counts after mid-2024, for informational articles, is consistent with more zero-click behavior.
  - **Shortcomings:** Aggregated; does not distinguish Google from other search engines; only captures traffic to Wikipedia, not to other sites; (referrer, resource) pairs with ≤10 observations are excluded.

- **Dataset #2 – Google Trends**
  - **Dataset Name:** Google Trends (interest over time)
  - **Link:** https://trends.google.com/ (or via `pytrends` in Python)
  - **Observations:** One row per date (or week) per keyword.
  - **Variables:** `date`, keyword (search term), `interest_over_time` (0–100, relative popularity).
  - **Relevance:** Shows whether people kept searching for the same topics (e.g. "Quantum mechanics", "Climate change") across 2023–2025. Stable or rising interest with falling Wikipedia clickstream counts would support that answers are being consumed in-search (zero-click).
  - **Shortcomings:** Relative scale (0–100), not absolute search volume; possible sampling/API limits; geographic and category choices affect results.

**Combining the datasets:** We will align both datasets by **time** (month) and **topic**. For each informational topic (e.g. a set of Wikipedia article titles and corresponding search terms), we will (1) compute monthly Wikipedia clickstream counts from search to those articles, and (2) get monthly Google Trends interest for the matching keywords. We will then compare pre– vs post–AI Overviews periods (e.g. 2023 vs 2024–2025) and test whether the ratio of clicks-to-Wikipedia to search-interest declines, which would be consistent with a shift toward zero-click behavior.

In [1]:
# Run this code every time when you're actively developing modules in .py files.  It's not needed if you aren't making modules
#
## this code is necessary for making sure that any modules we load are updated here 
## when their source code .py files are modified

%load_ext autoreload
%autoreload 2

### Wikipedia Clickstream (Dataset #1)

**Metrics and units:** Each row is a (referrer, resource) pair with count `n`. **prev**: referrer—either an article title (internal link) or a fixed code (e.g. `other-search` for external search engines, `other-external` for other external sites, `other-empty` for no referrer). **curr**: destination Wikipedia article (main namespace). **type**: `link` (prev links to curr), `external` (prev is external), or `other`. **n**: integer count of that (prev, curr) transition in that month. Pairs with 10 or fewer observations are excluded in the source. Counts are aggregated over desktop, mobile web, and mobile app.

**Relevance to the project:** Filtering rows where `prev == 'other-search'` gives monthly counts of clicks from search engines to Wikipedia articles. Summing `n` over selected informational articles (or over all articles) yields a proxy for “organic clicks from search to Wikipedia.” A drop in this total after AI Overviews (2024) would be consistent with more zero-click behavior.

**Concerns:** Data is aggregated and anonymized; we cannot separate Google from other search engines. Only Wikipedia traffic is observed, so we are measuring one slice of organic clicks. Low-count pairs are dropped at source. Referrer mapping may change over time (e.g. “other-search” aggregates all search engines).


In [6]:
import os
import pandas as pd
import glob

# Ensure directories exist
os.makedirs('data/01-interim', exist_ok=True)
os.makedirs('data/02-processed', exist_ok=True)

# Load all Wikipedia Clickstream monthly files from 00-raw
raw_dir = 'data/00-raw'
pattern = os.path.join(raw_dir, 'clickstream-enwiki-*.tsv.gz')
files = sorted(glob.glob(pattern))

if not files:
    raise FileNotFoundError("No clickstream files found in data/00-raw/. Run the setup cell above to download them.")

# Load and parse each monthly file; add month column
dfs = []
for path in files:
    # e.g. clickstream-enwiki-2024-06.tsv.gz -> 2024-06
    print(f"Loading {path} ...")
    basename = os.path.basename(path)
    month = basename.replace('clickstream-enwiki-', '').replace('.tsv.gz', '')
    df = pd.read_csv(
        path, sep='\t', compression='gzip',
        header=None, names=['prev', 'curr', 'type', 'n'],
        on_bad_lines='skip'
    )
    df['month'] = month
    dfs.append(df)

clickstream_raw = pd.concat(dfs, ignore_index=True)

print("Shape:", clickstream_raw.shape)
print("Columns:", clickstream_raw.columns.tolist())
print("\nFirst few rows:")
display(clickstream_raw.head(10))
print("\nValue counts for 'prev' (top referrers):")
print(clickstream_raw['prev'].value_counts().head(15))


Loading data/00-raw/clickstream-enwiki-2023-06.tsv.gz ...
Loading data/00-raw/clickstream-enwiki-2024-06.tsv.gz ...
Loading data/00-raw/clickstream-enwiki-2025-01.tsv.gz ...
Shape: (104239331, 5)
Columns: ['prev', 'curr', 'type', 'n', 'month']

First few rows:


,prev,curr,type,n,month
0,other-empty,Kensey_Johns_Jr.,external,32.0,2023-06
1,other-empty,Tengah_Islands,external,14.0,2023-06
2,other-search,Kensey_Johns_Jr.,external,17.0,2023-06
3,other-search,Tengah_Islands,external,10.0,2023-06
4,List_of_number-one_hits_of_1973_(Mexico),List_of_number-one_hits_of_1974_(Mexico),link,16.0,2023-06
5,other-empty,1957–58_British_Home_Championship,external,27.0,2023-06
6,other-empty,List_of_number-one_hits_of_1974_(Mexico),external,10.0,2023-06
7,British_Home_Championship,1957–58_British_Home_Championship,link,10.0,2023-06
8,other-search,List_of_number-one_hits_of_1974_(Mexico),external,43.0,2023-06
9,other-empty,Sensory_history,external,156.0,2023-06



Value counts for 'prev' (top referrers):
prev
other-empty                                                      12766080
other-search                                                     11725128
other-internal                                                    4467687
other-external                                                    1641561
Main_Page                                                          806020
other-other                                                        515407
Wikipedia                                                           25684
Wiki                                                                 9139
Deaths_in_2023                                                       5956
United_States                                                        5244
Deaths_in_2024                                                       5147
List_of_Korean_dramas                                                5108
List_of_one-hit_wonders_in_the_United_States                     

In [7]:
# Missingness
print("Missing per column:")
print(clickstream_raw.isnull().sum())
# Clickstream dumps are complete for (prev, curr) pairs; no NA expected in prev, curr, n.

# Restrict to search-originated traffic: prev == 'other-search' (external search engines)
clickstream_search = clickstream_raw[clickstream_raw['prev'] == 'other-search'].copy()
print("\nRows with prev='other-search':", len(clickstream_search))

# Tidy: one row per (month, curr) with total clicks from search to that article
clickstream_by_month = (
    clickstream_search.groupby(['month', 'curr'], as_index=False)['n']
    .sum()
    .rename(columns={'n': 'clicks_from_search'})
)
# Total clicks from search per month (for trend analysis)
monthly_totals = clickstream_by_month.groupby('month', as_index=False)['clicks_from_search'].sum()
print("\nTotal clicks from search by month:")
print(monthly_totals)

# Save processed clickstream: search-only, by month and article
clickstream_by_month.to_csv('data/02-processed/wikipedia_clickstream_search_by_month_article.csv', index=False)
monthly_totals.to_csv('data/02-processed/wikipedia_clickstream_search_monthly_totals.csv', index=False)
print("\nSaved to data/02-processed/")

Missing per column:
prev     108
curr     172
type       0
n         24
month      0
dtype: int64

Rows with prev='other-search': 11725128

Total clicks from search by month:
     month  clicks_from_search
0  2023-06        3.123554e+09
1  2024-06        2.955755e+09
2  2025-01        3.319773e+09

Saved to data/02-processed/


### Google Trends (Dataset #2)

**Metrics and units:** Google Trends provides **interest over time**: a 0–100 index of relative search popularity for a given keyword in a given time window (no absolute volume). We use **date** (or weekly buckets) and **interest** (0–100) per keyword. This tells us whether people were still searching for the same topics across 2023–2025.

**Relevance:** If Trends interest stays stable or rises while Wikipedia clickstream counts from search drop, that supports the hypothesis that more searches are “zero-click” (answers consumed in-search rather than by clicking through to Wikipedia).

**Concerns:** Relative scale only; geographic and category filters affect results; API/rate limits may apply when using `pytrends`; data is sampled.

**Obtaining the data:** We use the `pytrends` library to fetch interest over time for chosen keywords (e.g. "Quantum mechanics", "Climate change"). Below we load or fetch a small example and tidy it; for the full analysis we will fetch multiple keywords and align by month with the clickstream data.

In [10]:
# Google Trends: fetch via pytrends (install with: pip install pytrends)
# If pytrends is not available, we can load a pre-downloaded CSV from 00-raw instead.

try:
    from pytrends.request import TrendReq
    pytrends_available = True
except ImportError:
    pytrends_available = False
    print("pytrends not installed. Install with: pip install pytrends")
    print("Alternatively, export CSVs from https://trends.google.com/ and load from data/00-raw/")

if pytrends_available:
    pt = TrendReq(hl='en-US', tz=360)
    # Example: interest over time for a few informational keywords (2023-01-01 to 2025-02-01)
    keywords = ["Quantum mechanics", "Climate change"]
    pt.build_payload(keywords, timeframe='2023-01-01 2025-02-01', geo='')
    trends_df = pt.interest_over_time()
    if 'isPartial' in trends_df.columns:
        trends_df = trends_df.drop(columns=['isPartial'])
    trends_df = trends_df.reset_index()
    trends_df = trends_df.rename(columns={'date': 'date'})
    # Tidy: long form (date, keyword, interest)
    trends_long = trends_df.melt(id_vars=['date'], var_name='keyword', value_name='interest')
    trends_long['month'] = trends_long['date'].dt.to_period('M').astype(str)
    print("Shape:", trends_long.shape)
    print(trends_long.head(10))
    # Save processed Trends data
    trends_long.to_csv('data/02-processed/google_trends_interest_long.csv', index=False)
    print("\nSaved to data/02-processed/google_trends_interest_long.csv")
else:
    # Placeholder: create minimal structure so rest of notebook can run
    import pandas as pd
    trends_long = pd.DataFrame({'month': [], 'keyword': [], 'interest': []})
    print("Google Trends data will be loaded from CSV or fetched once pytrends is installed.")


Shape: (218, 4)
        date            keyword  interest    month
0 2023-01-01  Quantum mechanics         0  2023-01
1 2023-01-08  Quantum mechanics         0  2023-01
2 2023-01-15  Quantum mechanics         0  2023-01
3 2023-01-22  Quantum mechanics         0  2023-01
4 2023-01-29  Quantum mechanics         0  2023-01
5 2023-02-05  Quantum mechanics         0  2023-02
6 2023-02-12  Quantum mechanics         0  2023-02
7 2023-02-19  Quantum mechanics         0  2023-02
8 2023-02-26  Quantum mechanics         0  2023-02
9 2023-03-05  Quantum mechanics         0  2023-03

Saved to data/02-processed/google_trends_interest_long.csv


### Combining the two datasets

We align Wikipedia Clickstream and Google Trends by **month**. For each month we have (1) total (or topic-specific) clicks from search to Wikipedia from the clickstream data, and (2) average or total interest per keyword from Trends. We can then compute a simple ratio or normalized metric (e.g. clicks per unit of interest) and compare pre–AI Overviews (e.g. 2023) vs post–AI Overviews (2024–2025). Below we load the processed files and join on `month` for illustration.

In [11]:
# Load processed data and align by month
monthly_totals = pd.read_csv('data/02-processed/wikipedia_clickstream_search_monthly_totals.csv')
monthly_totals = monthly_totals.rename(columns={'clicks_from_search': 'wiki_clicks_from_search'})

# If we have Trends data, average interest by month and join
trends_path = 'data/02-processed/google_trends_interest_long.csv'
if os.path.exists(trends_path) and os.path.getsize(trends_path) > 0:
    trends = pd.read_csv(trends_path)
    trends['date'] = pd.to_datetime(trends['date'])
    trends['month'] = trends['date'].dt.to_period('M').astype(str)
    trends_monthly = trends.groupby('month', as_index=False)['interest'].mean()
    combined = monthly_totals.merge(trends_monthly, on='month', how='outer')
    print("Combined dataset (by month):")
    display(combined)
else:
    print("Wikipedia monthly totals (Trends will be joined when available):")
    display(monthly_totals)

Combined dataset (by month):


,month,wiki_clicks_from_search,interest
0,2023-01,NaN,1.900
1,2023-02,NaN,2.125
2,2023-03,NaN,2.500
3,2023-04,NaN,8.300
4,2023-05,NaN,2.375
5,2023-06,3.123554e+09,1.750
6,2023-07,NaN,1.600
7,2023-08,NaN,1.625
8,2023-09,NaN,2.000
9,2023-10,NaN,2.000


## Ethics

For each item: put an X if considered; if relevant, add a short paragraph. See [Deon checklist examples](https://deon.drivendata.org/examples/).

**A. Data Collection**
- [X] **A.1 Informed consent:** We use clickstream and SEO-style data from platforms (e.g. Wikimedia, Google Trends) that publish aggregated, anonymized data. We only use sources that state consent/anonymization practices.
- [X] **A.2 Collection bias:** We acknowledge bias (e.g. tech-savvy users, desktop vs mobile). We stratify by device where possible and state sampling limitations.
- [X] **A.3 Limit PII exposure:** We use only aggregated, anonymized data (no individual search histories or PII).
- [ ] **A.4 Downstream bias mitigation:** Not applicable (no protected group status collected).

**B. Data Storage**
- [ ] **B.1 Data security:** Plan to protect data (e.g. access controls, no raw data in public repo if sensitive).
- [ ] **B.2 Right to be forgotten:** N/A for aggregate public datasets.
- [ ] **B.3 Data retention plan:** Plan to delete or archive after project if required.

**C. Analysis**
- [ ] **C.1 Missing perspectives:** We will consider stakeholder blindspots where relevant.
- [X] **C.2 Dataset bias:** We address bias (e.g. commercial/English-language emphasis) by defining scope to informational queries and stating limitations.
- [X] **C.3 Honest representation:** Visualizations and statistics will represent the data honestly.
- [X] **C.4 Privacy in analysis:** No PII used or displayed.
- [X] **C.5 Auditability:** Process documented in Jupyter notebooks; code in GitHub for reproducibility.

**D. Modeling / E. Deployment:** Addressed in later checkpoints if we deploy models.

## Team Expectations 

- **Communication:** Primarily via iMessage group chat; meet via FaceTime or in person as needed. Respond within 24 hours; notify in advance if unable to attend a meeting.
- **Equal contribution:** Each member contributes equally across research, coding, writing, and editing; we rotate responsibilities.
- **Tone and respect:** Blunt but polite; use "I statements" when giving feedback; assume criticism is well-intentioned.
- **Task management:** Use GitHub for tasks and deadlines; assign fairly by strengths and availability. If struggling, notify within 48 hours to redistribute.
- **Decision making:** Majority vote for major decisions; for urgent decisions when someone is unresponsive, available members can proceed and update afterward.
- **Accountability:** If someone is not meeting expectations, address directly (e.g. via text) with one week to improve and specific deliverables; redistribute tasks as needed.
- **Deadlines:** Internal deadlines 2–3 days before official course deadlines for review and revision.

## Project Timeline Proposal

**Special resources/training:** Statistical methods for pre/post AI Overviews comparison (e.g. t-tests, chi-square); clear visualizations for trends over time.

| Meeting Date | Meeting Time | Completed Before Meeting | Discuss at Meeting |
|--------------|--------------|--------------------------|-------------------|
| 1/15 | 12 PM | NA | Determine best form of communication |
| 1/23 | 10 AM | Brainstorm topics (All) | Brainstorm topics (All) |
| 1/26 | 10 AM | Background research (Ali, Rodayna); Ethics draft (Sabine) | Ideal dataset(s) and ethics; draft proposal |
| 2/4 | 10 AM | Edit, finalize, submit proposal (Maanav, Tessa); Search datasets (All) | Wrangling and analysis approaches; assign leads |
| 2/4 | Before 11:59 PM | NA | **Turn in Project Proposal** |
| 2/14 | 6 PM | Import & wrangle data (Maanav); Initial EDA with 3+ visualizations (Rodayna) | Review wrangling/EDA; analysis plan; data quality |
| 2/14 | Before 11:59 PM | NA | **Turn in Data Checkpoint** |
| 2/23 | 12 PM | Finalize wrangling/EDA (Maanav, Rodayna); Begin pre/post statistical analysis (Sabine, Tessa) | Edit analysis; preliminary results; check-in |
| 3/4 | Before 11:59 PM | NA | **Turn in EDA Checkpoint** |
| 3/9 | 12 PM | Finalize analysis and visualizations; Draft results/conclusion (Ali); Polish notebook (Maanav) | Review analysis; edit results/discussion; video outline |
| 3/13 | 12 PM | Ethics updates (Sabine); Finalize written sections (Ali, Tessa); Record video segments (All) | Integrate video; final review |
| 3/18 | Before 11:59 PM | NA | **Turn in Final Project & Video** |

*Timeline updated for Data Checkpoint: data sources set (Wikipedia Clickstream + Google Trends); wrangling and combination by month in progress.*